In [47]:
from data_generator.csv_data_processor import CSVDataProcessor
from utils.util_funcs import get_row_count, display_df, get_column_names
from utils.read_write import read_postgres_table
from spark_instance import spark
from pyspark.sql import Window
from data_generator.constants import ColConstants

from pyspark.sql.functions import col, lower, lit, rand, array, floor, concat, when, row_number, concat_ws, date_format, count
  

In [2]:
# csv_reader = CSVDataProcessor(spark, "data/healthcare_dataset.csv")
# 
# # Read the CSV file
# raw_df = csv_reader.run()


In [3]:
# min_age_days = 1 * 365  # Minimum age in days (18 years)
# max_age_days = 90 * 365  # Maximum age in days (90 years)
# 
# raw_df = (raw_df.withColumn("DOB", expr(f"date_sub(current_date(), CAST(round(rand() * ({max_age_days} - {min_age_days}) + {min_age_days}) AS INT))"))
#         .withColumn("Age", floor(datediff(current_date(), col("DOB")) / 365.25)))




In [4]:
df = read_postgres_table("dob_age_raw_data")

In [5]:
# df.show()
get_row_count(df, True)

10000


10000

In [6]:
# df.where(col("Age") < 16).show()

In [7]:
from constants.admission_types_tests_dataset import admission_mapping, admission_tests
# Flatten the mapping and create a DataFrame
flattened = [
    (top_level, sub_level, stay_type, admission_tests.get(sub_level, ["No tests"]))
    for top_level, sub_level_dict in admission_mapping.items()
    for sub_level, stay_types in sub_level_dict.items()
    for stay_type in stay_types
]



In [8]:
"""
STEP 1 
    From the flattened data list we wish to create a DataFrame. This contains all the possible combinations for the given 
    top level admissions, sub level admissions, stay types and list of tests available from the admission_mapping, stay_type and admission_tests lists or dictionaries in admission_types_test_dataset.py
"""
mapping_df = spark.createDataFrame(flattened, ["top_level_admission", "sub_level_admission", "stay_type", "possible_tests"])

display_df(mapping_df, 100)

,top_level_admission,sub_level_admission,stay_type,possible_tests
0,emergency,injury_rtc,inpatient,"[X-rays, CT scans, MRI, Ultrasound, Blood tests]"
1,emergency,injury_rtc,day_patient,"[X-rays, CT scans, MRI, Ultrasound, Blood tests]"
2,emergency,self_inflicted,inpatient,"[Psychological assessment, X-rays (for physica..."
3,emergency,cardiology,inpatient,"[ECG, Echocardiogram, Stress tests, Cardiac ca..."
4,emergency,cardiology,day_patient,"[ECG, Echocardiogram, Stress tests, Cardiac ca..."
5,emergency,neurology,inpatient,"[MRI or CT scans of the brain, Electroencephal..."
6,emergency,pulmonology,inpatient,"[Pulmonary function tests, Chest X-ray, CT sca..."
7,emergency,pulmonology,day_patient,"[Pulmonary function tests, Chest X-ray, CT sca..."
8,emergency,infectious_diseases,inpatient,"[Blood cultures, PCR tests, Antibody tests, ch..."
9,emergency,infectious_diseases,outpatient,"[Blood cultures, PCR tests, Antibody tests, ch..."


In [9]:
"""
STEP 2 
    Create a list of admission_types randomly assign this to the original patient in the original data set, 
    whilst dropping the original admission_type column. Then join with mapping_df on top_level_admission col to give access to possible 
    conditions, mappings and so on.
"""
admission_types = list(admission_mapping.keys())

print(admission_types)

keys_array = array([lit(key) for key in admission_types])

# Define constants and conditions
female_only = ['maternity', 'obstetrics']
is_female = lower(col('gender')) == 'female'
is_pediatric = col("Age") < 18
is_geriatric = (col("Age") >= 65) & (col("sub_level_admission") == "geriatrics")

df = (df.withColumn("top_level_admission", keys_array[floor(rand() * len(admission_types))]).drop("admission_type")
        .withColumn("is_female", is_female)
        .withColumn("is_pediatric", is_pediatric)
      )
get_row_count(df, True)
df = df.join(mapping_df, "top_level_admission", "inner")
get_row_count(df, True)
display_df(df, 999)

['emergency', 'gp_referral', 'hospital_referral', 'self_referral', 'elective']
10000
90008


,top_level_admission,name,Age,gender,blood_type,medical_condition,date_of_admission,doctor,hospital,insurance_provider,...,room_number,discharge_date,medication,test_results,DOB,is_female,is_pediatric,sub_level_admission,stay_type,possible_tests
0,elective,Tiffany Ramirez,39,Female,O-,Diabetes,2022-11-17,Patrick Parker,Wallace-Hamilton,Medicare,...,146,2022-12-01,Aspirin,Inconclusive,1984-09-30,True,False,maternity,inpatient,"[Ultrasound, Blood tests, Glucose tolerance te..."
1,elective,Tiffany Ramirez,39,Female,O-,Diabetes,2022-11-17,Patrick Parker,Wallace-Hamilton,Medicare,...,146,2022-12-01,Aspirin,Inconclusive,1984-09-30,True,False,maternity,day_patient,"[Ultrasound, Blood tests, Glucose tolerance te..."
2,elective,Tiffany Ramirez,39,Female,O-,Diabetes,2022-11-17,Patrick Parker,Wallace-Hamilton,Medicare,...,146,2022-12-01,Aspirin,Inconclusive,1984-09-30,True,False,orthopedics,inpatient,"[X-rays, MRI, CT scans, Bone scans, Blood tests]"
3,elective,Tiffany Ramirez,39,Female,O-,Diabetes,2022-11-17,Patrick Parker,Wallace-Hamilton,Medicare,...,146,2022-12-01,Aspirin,Inconclusive,1984-09-30,True,False,orthopedics,day_patient,"[X-rays, MRI, CT scans, Bone scans, Blood tests]"
4,elective,Tiffany Ramirez,39,Female,O-,Diabetes,2022-11-17,Patrick Parker,Wallace-Hamilton,Medicare,...,146,2022-12-01,Aspirin,Inconclusive,1984-09-30,True,False,obstetrics,inpatient,"[Ultrasound, Blood tests, Glucose tolerance te..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,elective,David Boyd,25,Female,AB-,Cancer,2021-03-07,Kyle Miller,"Caldwell, Patton and Scott",Medicare,...,355,2021-04-05,Penicillin,Abnormal,1998-06-10,True,False,obstetrics,inpatient,"[Ultrasound, Blood tests, Glucose tolerance te..."
995,elective,David Boyd,25,Female,AB-,Cancer,2021-03-07,Kyle Miller,"Caldwell, Patton and Scott",Medicare,...,355,2021-04-05,Penicillin,Abnormal,1998-06-10,True,False,ophthalmology,day_patient,"[Eye exam, Tonometry (eye pressure test), Reti..."
996,elective,David Boyd,25,Female,AB-,Cancer,2021-03-07,Kyle Miller,"Caldwell, Patton and Scott",Medicare,...,355,2021-04-05,Penicillin,Abnormal,1998-06-10,True,False,ophthalmology,outpatient,"[Eye exam, Tonometry (eye pressure test), Reti..."
997,elective,David Boyd,25,Female,AB-,Cancer,2021-03-07,Kyle Miller,"Caldwell, Patton and Scott",Medicare,...,355,2021-04-05,Penicillin,Abnormal,1998-06-10,True,False,geriatrics,inpatient,"[Comprehensive geriatric assessment, Blood tes..."


In [12]:
"""
STEP 3 
    Create a rowq ranking using a unique cols, created from stay_name and unique_id. 
    Verification done below. 
    
"""

df =  df.withColumn('stay_name', 
                   when(col('stay_type') == 'out_patient', concat(col('name'), lit('_out_patient')))
                   .when(col('stay_type') == 'inpatient', concat(col('name'), lit('_inpatient')))
                   .when(col('stay_type') == 'day_patient', concat(col('name'), lit('_day_patient')))
                   .otherwise(col('name'))
                  ).withColumn("unique_id", concat_ws("_", "name", date_format("DOB", "yyyyMMdd")))


# Define a window specification that partitions data by 'top_level_admission' (or another unique patient identifier if needed)
windowSpec = Window.partitionBy('stay_name', 'unique_id').orderBy(rand())

# Assign row numbers within each partition in a random order
ranked_df = df.withColumn("row_num", row_number().over(windowSpec))


In [53]:
ranked_df.createGlobalTempView("ranked_df")

unique_dobs_df = spark.sql("""
WITH NameCounts AS (
    SELECT name
    FROM global_temp.ranked_df
)

SELECT DISTINCT r.name, r.unique_id
FROM global_temp.ranked_df r
JOIN NameCounts n ON r.name = n.name
ORDER BY r.name
LIMIT 10
""")

In [54]:
from pyspark.sql import DataFrame

# COSTLY WAY TO VERIFY THAT THE RANK WORKS BELOW.... 

def verify_ranking_counts(_df: DataFrame, ranked_df: DataFrame, names: list, unique_ids: list):
    for name, unique_id in zip(names, unique_ids):
        # Define filter conditions
        name_condition = col("name") == name
        unique_id_condition = col("unique_id") == unique_id
        stay_name_condition = col("stay_name").like(f"{name}_%")

        df_filtered_count = (_df
                             .filter(name_condition & unique_id_condition & stay_name_condition)
                             .count())

        ranked_df_filtered_count = (ranked_df
                                    .filter(name_condition & unique_id_condition & stay_name_condition)
                                    .count())

        # Assert that the counts are equal for each name-unique_id pair
        assert df_filtered_count == ranked_df_filtered_count, f"The number of matching rows for {name} with unique ID {unique_id} should be the same in both DataFrames."

# Example usage:
unique_names = [row['name'] for row in unique_dobs_df.select("name").collect()]
unique_ids = [row['unique_id'] for row in unique_dobs_df.select("unique_id").collect()]
verify_ranking_counts(df, ranked_df, unique_names, unique_ids)

In [52]:
def verify_ranking(_df: DataFrame, ranked_df: DataFrame):
    joined_df = _df.alias("df").join(
        ranked_df.alias("ranked"),
       
       (col("df.unique_id") == col("ranked.unique_id")),
        "inner"
    )

    verification_df = joined_df.select(
       col("df.unique_id")
    ).distinct().groupBy("unique_id").agg(
        count("unique_id").alias("unique_id_count")
    )

    # Check if any name has a mismatch in the count of unique IDs
    mismatch_count = verification_df.filter("unique_id_count > 1").count()

    # Assert that there are no mismatches
    assert mismatch_count == 0, f"There are names with mismatched unique ID counts. mismatch_count: {mismatch_count}"

verify_ranking(df, ranked_df)


0


In [60]:
"""
Step 4: 
    Sort out geriatrics data and verify that individuals are not geriatrics 
"""

not_geriatric_df = ranked_df.withColumn("is_geriatric", is_geriatric).where((col('sub_level_admission') == 'geriatrics') &( col("is_geriatric") == False))

ranked_tot = get_row_count(ranked_df)
geriatric_tot = get_row_count(not_geriatric_df)

filtered_df = ranked_df.join(not_geriatric_df, on= ['sub_level_admission', 'DOB'], how="left_anti")

filtered_tot = get_row_count(filtered_df)

assert filtered_tot == ranked_tot - geriatric_tot, "Row counts do not match expected value"

# checks the above calculation
assert get_row_count(filtered_df.where((col('sub_level_admission') == 'geriatrics') &( col("is_geriatric") == False))) == 0

In [ ]:
# TODO filter on is pediatric, geriatric and is_female to be done here and same people with dob? needs  to be considered 

In [70]:
assert get_row_count(ranked_df.where((col("gender") == "Male") & (col("is_female") == True))) == 0

not_female_df = filtered_df.where((col("sub_level_admission").isin(female_only)) & (col("is_female") == False))

assert  not_female_df.where((col("gender") == "Female") & (col("is_female") == True)).count() == 0

In [77]:
display_df(not_female_df)

,sub_level_admission,DOB,top_level_admission,name,Age,gender,blood_type,medical_condition,date_of_admission,doctor,...,medication,test_results,is_female,is_pediatric,stay_type,possible_tests,stay_name,unique_id,is_geriatric,row_num
0,maternity,1978-06-05,elective,Aaron Hamilton,45,Male,B-,Hypertension,2022-08-21,Traci Williams,...,Ibuprofen,Abnormal,False,False,day_patient,"[Ultrasound, Blood tests, Glucose tolerance te...",Aaron Hamilton_day_patient,Aaron Hamilton_19780605,False,3
1,maternity,1978-06-05,elective,Aaron Hamilton,45,Male,B-,Hypertension,2022-08-21,Traci Williams,...,Ibuprofen,Abnormal,False,False,inpatient,"[Ultrasound, Blood tests, Glucose tolerance te...",Aaron Hamilton_inpatient,Aaron Hamilton_19780605,False,1
2,obstetrics,1978-06-05,elective,Aaron Hamilton,45,Male,B-,Hypertension,2022-08-21,Traci Williams,...,Ibuprofen,Abnormal,False,False,inpatient,"[Ultrasound, Blood tests, Glucose tolerance te...",Aaron Hamilton_inpatient,Aaron Hamilton_19780605,False,3
3,maternity,1995-08-24,elective,Aaron Hughes,28,Male,O-,Asthma,2020-12-04,Scott Beck,...,Aspirin,Abnormal,False,False,day_patient,"[Ultrasound, Blood tests, Glucose tolerance te...",Aaron Hughes_day_patient,Aaron Hughes_19950824,False,3
4,obstetrics,1995-08-24,elective,Aaron Hughes,28,Male,O-,Asthma,2020-12-04,Scott Beck,...,Aspirin,Abnormal,False,False,inpatient,"[Ultrasound, Blood tests, Glucose tolerance te...",Aaron Hughes_inpatient,Aaron Hughes_19950824,False,1
5,maternity,1995-08-24,elective,Aaron Hughes,28,Male,O-,Asthma,2020-12-04,Scott Beck,...,Aspirin,Abnormal,False,False,inpatient,"[Ultrasound, Blood tests, Glucose tolerance te...",Aaron Hughes_inpatient,Aaron Hughes_19950824,False,2
6,maternity,1983-07-07,elective,Aaron Miller,40,Male,A+,Arthritis,2020-09-27,Matthew Mora,...,Paracetamol,Normal,False,False,day_patient,"[Ultrasound, Blood tests, Glucose tolerance te...",Aaron Miller_day_patient,Aaron Miller_19830707,False,3
7,obstetrics,1983-07-07,elective,Aaron Miller,40,Male,A+,Arthritis,2020-09-27,Matthew Mora,...,Paracetamol,Normal,False,False,inpatient,"[Ultrasound, Blood tests, Glucose tolerance te...",Aaron Miller_inpatient,Aaron Miller_19830707,False,1
8,maternity,1983-07-07,elective,Aaron Miller,40,Male,A+,Arthritis,2020-09-27,Matthew Mora,...,Paracetamol,Normal,False,False,inpatient,"[Ultrasound, Blood tests, Glucose tolerance te...",Aaron Miller_inpatient,Aaron Miller_19830707,False,3
9,maternity,1951-07-03,elective,Abigail Horn,72,Male,AB+,Diabetes,2021-10-26,Elizabeth Flowers,...,Paracetamol,Normal,False,False,day_patient,"[Ultrasound, Blood tests, Glucose tolerance te...",Abigail Horn_day_patient,Abigail Horn_19510703,False,3


In [88]:
not_female_tot = get_row_count(not_female_df)

filtered_df_female = filtered_df.join(not_female_df, on=['sub_level_admission', 'DOB'], how = 'left_anti')

filtered_female_tot = get_row_count(filtered_df_female, True)

assert filtered_df_female.where((col("gender") == "Male") & (col("sub_level_admission").isin(female_only))).count() == 0 


81354


In [91]:
# filter pediatric patients who cannot be pregnant (based on legal age in the UK, 16) No assumption made an individual cannot choose to get pregnant before this age. 
under16_female_df = filtered_df_female.where((col("Age") <16) & (col("sub_level_admission") == 'maternity')).orderBy("Age")

In [98]:
filtered_df_female_u16 = filtered_df_female.join(under16_female_df, on=['sub_level_admission', 'DOB'], how = 'left_anti')

filtered_female_tot_u16 = get_row_count(under16_female_df, True)

assert filtered_df_female_u16.where((col("sub_level_admission") == "maternity") & (col("Age") <16)).count() == 0 

,sub_level_admission,DOB,top_level_admission,name,Age,gender,blood_type,medical_condition,date_of_admission,doctor,...,medication,test_results,is_female,is_pediatric,stay_type,possible_tests,stay_name,unique_id,is_geriatric,row_num
0,maternity,2023-02-04,elective,Angela Brown,1,Female,B+,Obesity,2019-02-28,Kyle Wu,...,Ibuprofen,Abnormal,True,True,day_patient,"[Ultrasound, Blood tests, Glucose tolerance te...",Angela Brown_day_patient,Angela Brown_20230204,False,2
1,maternity,2023-02-04,elective,Angela Brown,1,Female,B+,Obesity,2019-02-28,Kyle Wu,...,Ibuprofen,Abnormal,True,True,inpatient,"[Ultrasound, Blood tests, Glucose tolerance te...",Angela Brown_inpatient,Angela Brown_20230204,False,2
2,maternity,2022-05-13,elective,Cindy Robinson,1,Female,B+,Asthma,2023-03-11,Ashley Dean,...,Paracetamol,Abnormal,True,True,day_patient,"[Ultrasound, Blood tests, Glucose tolerance te...",Cindy Robinson_day_patient,Cindy Robinson_20220513,False,3
3,maternity,2022-05-13,elective,Cindy Robinson,1,Female,B+,Asthma,2023-03-11,Ashley Dean,...,Paracetamol,Abnormal,True,True,inpatient,"[Ultrasound, Blood tests, Glucose tolerance te...",Cindy Robinson_inpatient,Cindy Robinson_20220513,False,2
4,maternity,2022-06-23,elective,Dustin Garrett,1,Female,O-,Diabetes,2023-07-06,Rodney Johnson,...,Ibuprofen,Abnormal,True,True,day_patient,"[Ultrasound, Blood tests, Glucose tolerance te...",Dustin Garrett_day_patient,Dustin Garrett_20220623,False,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,maternity,2008-10-01,elective,Mr. John Wright,15,Female,B-,Arthritis,2022-04-06,Travis Norris,...,Penicillin,Abnormal,True,True,inpatient,"[Ultrasound, Blood tests, Glucose tolerance te...",Mr. John Wright_inpatient,Mr. John Wright_20081001,False,1
312,maternity,2008-08-19,elective,Robert Ali,15,Female,A+,Obesity,2020-04-26,Steven Bailey,...,Penicillin,Inconclusive,True,True,day_patient,"[Ultrasound, Blood tests, Glucose tolerance te...",Robert Ali_day_patient,Robert Ali_20080819,False,3
313,maternity,2008-08-19,elective,Robert Ali,15,Female,A+,Obesity,2020-04-26,Steven Bailey,...,Penicillin,Inconclusive,True,True,inpatient,"[Ultrasound, Blood tests, Glucose tolerance te...",Robert Ali_inpatient,Robert Ali_20080819,False,1
314,maternity,2008-11-29,elective,Steven Castillo,15,Female,A+,Arthritis,2019-09-18,Natasha Gray,...,Ibuprofen,Abnormal,True,True,day_patient,"[Ultrasound, Blood tests, Glucose tolerance te...",Steven Castillo_day_patient,Steven Castillo_20081129,False,3


,sub_level_admission,DOB,top_level_admission,name,Age,gender,blood_type,medical_condition,date_of_admission,doctor,...,medication,test_results,is_female,is_pediatric,stay_type,possible_tests,stay_name,unique_id,is_geriatric,row_num
0,orthopedics,1979-03-09,gp_referral,Aaron Gonzales,44,Female,B-,Hypertension,2019-09-10,Ian Edwards,...,Aspirin,Inconclusive,True,False,day_patient,"[X-rays, MRI, CT scans, Bone scans, Blood tests]",Aaron Gonzales_day_patient,Aaron Gonzales_19790309,False,1
1,orthopedics,1979-03-09,gp_referral,Aaron Gonzales,44,Female,B-,Hypertension,2019-09-10,Ian Edwards,...,Aspirin,Inconclusive,True,False,inpatient,"[X-rays, MRI, CT scans, Bone scans, Blood tests]",Aaron Gonzales_inpatient,Aaron Gonzales_19790309,False,3
2,dermatology,1953-12-22,gp_referral,Alan Ibarra,70,Male,O+,Arthritis,2020-03-26,Steven Rhodes,...,Lipitor,Normal,False,False,outpatient,"[Skin biopsy, Patch tests, Skin scrapings, Blo...",Alan Ibarra,Alan Ibarra_19531222,False,5
3,mental,1993-10-04,self_referral,Albert Dunn,30,Male,A+,Obesity,2021-01-22,Rita Payne,...,Paracetamol,Abnormal,False,False,outpatient,"[Psychological evaluation, Blood tests, Brain ...",Albert Dunn,Albert Dunn_19931004,False,1
4,cardiology,1946-06-24,emergency,Alexander Harris,77,Male,AB+,Diabetes,2023-03-30,Daniel Mosley,...,Penicillin,Abnormal,False,False,day_patient,"[ECG, Echocardiogram, Stress tests, Cardiac ca...",Alexander Harris_day_patient,Alexander Harris_19460624,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,mental,1996-07-27,gp_referral,Joseph Coleman,27,Male,O-,Asthma,2022-05-16,Kelly Cantrell,...,Paracetamol,Inconclusive,False,False,inpatient,"[Psychological evaluation, Blood tests, Brain ...",Joseph Coleman_inpatient,Joseph Coleman_19960727,False,3
196,nephrology,1968-10-25,hospital_referral,Joseph Daniels,55,Male,A-,Hypertension,2020-04-10,Stephen Melton,...,Penicillin,Inconclusive,False,False,inpatient,"[Blood tests (renal function tests), Urine tes...",Joseph Daniels_inpatient,Joseph Daniels_19681025,False,4
197,oncology,2017-12-26,hospital_referral,Joseph Perez,6,Male,A-,Hypertension,2022-12-03,Janet Jordan,...,Aspirin,Abnormal,False,True,day_patient,"[Biopsies, MRI, Ultrasound, Blood tests, Imagi...",Joseph Perez_day_patient,Joseph Perez_20171226,False,3
198,oncology,2017-12-26,hospital_referral,Joseph Perez,6,Male,A-,Hypertension,2022-12-03,Janet Jordan,...,Aspirin,Abnormal,False,True,inpatient,"[Biopsies, MRI, Ultrasound, Blood tests, Imagi...",Joseph Perez_inpatient,Joseph Perez_20171226,False,4


316


In [ ]:
# TODO: drop full name for initial and surname since nname column is poor. 
# adjust doctor column 
# medical condition 
# admission date to be checked again dob, 
# medication to be checked 
# choose random results 

# drop stay_name and unique_id 


In [ ]:
# Filter to keep only the top-ranked row within each partition
ranked_df = filtered_df_female

In [ ]:
ranked_df.where(col("name") == "Daniel Mccoy").show(n=8000)

In [ ]:
from pyspark.sql.types import StringType

df = filtered_df_female.select([col(c).cast(StringType()).alias(c) for c in df.columns])
df.write.csv('./temp_data/female/filtered_df_female.csv', mode = 'overwrite', header=True)

In [ ]:
# TODO initialise first name to make data more realistic for name columns i.e. Daniel Mccoy is seen a Male name but here its Female

In [ ]:
spark.stop()